In [1]:
import numpy as np
import pandas as pd

In [2]:
hea_22 = pd.read_stata('ch22o_EN_1.0p.dta')
rel_22 = pd.read_stata('cr22o_EN_1.0p.dta')
soc_22 = pd.read_stata('cs22o_EN_1.1p.dta')
fam_22 = pd.read_stata('cf22o_EN_1.0p.dta')
wor_22 = pd.read_stata('cw22o_EN_1.0p.dta')
per_22 = pd.read_stata('cp22n_EN_1.0p.dta')
val_22 = pd.read_stata('cv22n_EN_1.0p.dta')
ass_22 = pd.read_stata('ca22h_EN_1.0p.dta')
inc_22 = pd.read_stata('ci22o_EN_1.0p.dta')
hou_22 = pd.read_stata('cd22o_EN_1.0p.dta')

In [3]:
all_22 = [ass_22, hou_22, fam_22, hea_22, inc_22, per_22, soc_22, val_22, wor_22]

merged_22 = all_22[0]

for all in all_22[1:]:
    merged_22 = pd.merge(merged_22, all, on='nomem_encr', how='inner')

In [6]:
merged_22.head()

,nomem_encr,ca22h_m,ca22h001,ca22h002,ca22h003,ca22h082,ca22h078,ca22h005,ca22h006,ca22h007,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [4]:
m22_80nan = merged_22.columns[merged_22.isna().mean() > 0.8].tolist()
clean_22 = merged_22.drop(columns=m22_80nan, axis=1)
clean_22.head()

,nomem_encr,ca22h_m,ca22h003,ca22h078,ca22h005,ca22h006,ca22h007,ca22h008,ca22h009,ca22h010,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,unknown,yes,no,no,no,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [5]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, *columns):
    new_column_name = columns[-1]
    
    try:
        selected_columns = list(columns[:-1])
        numeric_columns = df[selected_columns].apply(pd.to_numeric, errors='coerce')
        df[new_column_name] = numeric_columns.mean(axis=1)
    except KeyError as e:
        print(f'KeyError: {e} not found in DataFrame columns.')
        df[new_column_name] = np.nan

    df.drop(columns=list(columns[:-1]), axis=1, inplace=True)

#replaces given columns with one containing their sums
def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

Concepts in health:
1. **Subjective health** (ch22o004 to sub_hea)
2. **Relative health** (ch22o005 to rel_hea)
3. Life expectancy
4. **Mental health** (all neg as men_hea_neg and and all pos as men_hea_pos)
5. Height, weight
6. **Chronical problems** (ch22o018 to disability)
7. Health problems
8. Impediment in labour
9. **Sex drugs alcohol*** (ch22o126 to smoker, ch22o133 to alcohol, all substances to subs)
10. Medication
11. Physical activity - only until 2018
12. Health improving behaviour - - only until 2018
13. Use of healthcare
14. Sight and hearing
15. Health insurance

In [6]:
#select categorical columns and convert to numeric
hea_22cat = hea_22.select_dtypes(include='category').columns
hea_22[hea_22cat] = hea_22[hea_22cat].apply(lambda col: col.astype('category').cat.codes)

#average for mental health and drugs
hea_toavg = [('ch22o011','ch22o014','ch22o012', 'men_hea_neg'),('ch22o013', 'ch22o015', 'men_hea_pos'), ('ch22o159', 'ch22o160', 'ch22o161', 'ch22o161', 'ch22o162', 'ch22o163', 'ch22o270', 'subs')]
for column_set in hea_toavg:
    average_columns(hea_22, *column_set)

#create a new df hea_22_con with selected columnns and new columns containing avgs (representing concepts)
hea_con = {'ch22o004': 'sub_hea', 'ch22o005': 'rel_hea', 'ch22o018': 'disability', 'ch22o020':'mob_issues', 'ch22o126':'smoker', 'ch22o133':'alcohol'}
hea_22.rename(columns=hea_con, inplace=True)

hea_22cols = ['men_hea_neg', 'men_hea_pos', 'subs', 'sub_hea', 'rel_hea', 'disability', 'mob_issues', 'smoker', 'alcohol']
hea_22_con = hea_22.loc[:, hea_22cols]

#ready
hea_22_con.head()

NameError: name 'hea_tocombine' is not defined

Concepts in religion and ethnicity:

1. Religious upbringing (cr23p135 to rel_upb)
2. Religious affiliation (cr23p144 to rel_aff)
3. Religiosity (cr23p162 to relig)
4. Religious orthodoxy
5. Nationality 
6. Origin (cr23p079 – cr23p087 to lang_ori, cr23p079 indicating Dutch or not)
7. Ethnic identification (cr23p164 to eth_id)
8. Language proficiency and use

In [ ]:
rel_con = {'cr23p135':'rel_upb', 'cr23p144':'rel_aff', 'cr23p162':'relig', 'cr23p079':'lang_ori', 'cr23p164':'eth_id'}


Concepts in social intergation and leisure:

1. Loneliness
2. Social contacts 
3. Satisfaction leisure (cs22o001 to lei_sat)
4. Social engagement
5. Volunteer aid
6. Time expenditure voluntary work
7. Cultural participation (sum cs22o494 - cs22o496 to cul_par)
8. Holidays (cs22o103 to ab_hol)
9. Sport (cs22o105 to spo_h)
10. Media usage

In [30]:
codebook_cul = {'0 times': 0, '1 time': 1, '2 to 3 times': 2, '4 to 11 times': 3, '12 times or more': 4}
codebook_lei = {'not at all satisfied':0, 'completely satisfied':10}
codebook_hol = {'not once':0, 'one time':1, 'two times':2, 'three times':3, 'four times':4, 'five times or more':'≥5'}
cul_sum = ['cs22o494', 'cs22o495', 'cs22o517', 'cs22o093', 'cs22o094', 'cs22o568', 'cs22o516', 'cs22o496']
soc_22[cul_sum] = soc_22[cul_sum].replace(codebook_cul)
soc_22['cs22o001'] = soc_22['cs22o001'].replace(codebook_lei)
soc_22['cs22o103'] = soc_22['cs22o103'].replace(codebook_hol)
soc_22['cul'] = soc_22[cul_sum].sum(axis=1)
soc_con = {'cs22o001':'lei_sat', 'cul':'cul_par', 'cs22o103':'ab_hol', 'cs22o105':'spo_h'}
selected_columns = list(soc_con.keys())
soc_22_con = soc_22.loc[:, selected_columns].rename(columns=soc_con)
    
print(soc_22_con)

     lei_sat  cul_par ab_hol  spo_h
0        8.0      1.0      0    3.0
1        7.0      0.0      1    NaN
2         10      2.0      2    NaN
3        4.0      0.0      1    NaN
4        9.0      6.0      0    NaN
...      ...      ...    ...    ...
5879     8.0      4.0      1    3.0
5880     7.0      5.0      2    6.0
5881     2.0      0.0      1    4.0
5882     7.0      3.0      3   10.0
5883     8.0      8.0      3    5.0

[5884 rows x 4 columns]
